# Tutorial Test

Running tutorial codes for preprocessing [link](https://www.nishika.com/competitions/32/topics/310)

本コンペは日次の野菜卸売価格の予測タスクですが、ここでは簡単のため、月次の予測タスクに読み替えてみます（日毎に予測値は変えない）<br>
本notebookでは前処理を行い、次のnotebookでは学習・推論を行います

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = "MS Gothic" # Use this font to avoid encoding error
import os

In [2]:
train_df = pd.read_csv('../data/train.csv')
print(train_df.shape)
train_df

(97782, 5)


,kind,date,amount,mode_price,area
0,だいこん,20051107,201445,735.0,千葉
1,だいこん,20051108,189660,840.0,千葉_各地_青森
2,だいこん,20051110,218166,735.0,千葉_各地_青森
3,だいこん,20051111,182624,682.5,千葉_青森
4,だいこん,20051112,220691,682.5,千葉_青森
...,...,...,...,...,...
97777,しめじ,20170526,17231,48.5,新潟_長野
97778,しめじ,20170527,17105,48.5,新潟_長野
97779,しめじ,20170529,12406,48.5,新潟_長野
97780,しめじ,20170530,12320,48.5,新潟_長野


In [3]:
test_df = pd.read_csv('../data/test.csv')
print(test_df.shape)
test_df

(320, 3)


,kind,date,area
0,だいこん,20220502,千葉_各地_青森
1,だいこん,20220506,千葉_各地_青森
2,だいこん,20220507,千葉_各地_青森
3,だいこん,20220509,千葉_各地_青森
4,だいこん,20220510,千葉_各地_青森
...,...,...,...
315,ミニトマト,20220526,各地_愛知
316,ミニトマト,20220527,各地_愛知
317,ミニトマト,20220528,各地_愛知
318,ミニトマト,20220530,各地_愛知


In [4]:
wea_df = pd.read_csv('../data/weather.csv')
print(wea_df.shape)
wea_df

(204320, 10)


,date,mean_temp,max_temp,max_temp_time,min_temp,min_temp_time,sum_rain,sun_time,mean_humid,area
0,20041106,13.1,20.1,2004/11/6 12:50,8.1,2004/11/6 05:31,5.0,1.6,77.0,青森
1,20041107,9.6,12.5,2004/11/7 13:40,5.4,2004/11/7 22:17,0.0,4.1,63.0,青森
2,20041108,9.0,15.5,2004/11/8 12:51,3.2,2004/11/8 06:28,0.0,8.9,72.0,青森
3,20041109,12.2,17.7,2004/11/9 14:40,5.7,2004/11/9 02:49,1.5,1.2,85.0,青森
4,20041110,11.6,16.4,2004/11/10 14:42,6.9,2004/11/10 06:23,0.0,7.5,89.0,青森
...,...,...,...,...,...,...,...,...,...,...
204315,20220426,21.3,23.6,2022/4/26 14:27,17.9,2022/4/26 04:17,2.5,0.0,89.0,東京
204316,20220427,23.0,27.7,2022/4/27 13:12,18.9,2022/4/28 0:00,0.5,1.6,79.0,東京
204317,20220428,16.9,21.1,2022/4/28 15:03,14.2,2022/4/28 22:40,0.0,5.5,57.0,東京
204318,20220429,12.9,16.4,2022/4/29 09:21,9.0,2022/4/29 23:55,40.0,0.2,86.0,東京


まず、天候データを加工します。月毎の基本統計量をとります

In [5]:
wea_df['year'] = wea_df['date'].apply(lambda x: int(str(x)[:4]))
wea_df['month'] = wea_df['date'].apply(lambda x: int(str(x)[4:6]))

In [6]:
agg_cols = ['mean_temp', 'max_temp', 'min_temp', 'sum_rain', 'sun_time', 'mean_humid']
gb_df = wea_df.groupby(['area', 'year', 'month'])[agg_cols].agg(['mean','max','min']).reset_index()

new_cols = []
for col1, col2 in gb_df.columns:
    if col2:
        new_cols.append(col2+'_'+col1)
    else:
        new_cols.append(col1)

gb_df.columns = new_cols
gb_df

,area,year,month,mean_mean_temp,max_mean_temp,min_mean_temp,mean_max_temp,max_max_temp,min_max_temp,mean_min_temp,...,min_min_temp,mean_sum_rain,max_sum_rain,min_sum_rain,mean_sun_time,max_sun_time,min_sun_time,mean_mean_humid,max_mean_humid,min_mean_humid
0,仙台,2004,11,11.256000,16.4,4.4,15.196000,19.3,7.5,7.152000,...,1.1,1.340000,20.0,0.0,4.436000,9.4,0.0,68.320000,93.0,45.0
1,仙台,2004,12,5.422581,13.3,-1.2,9.212903,21.6,0.4,1.825806,...,-4.4,2.048387,42.0,0.0,4.441935,8.9,0.0,64.225806,86.0,46.0
2,仙台,2005,1,1.590323,5.4,-2.0,5.077419,11.1,0.6,-1.809677,...,-5.6,2.000000,44.5,0.0,3.938710,9.4,0.0,69.709677,92.0,51.0
3,仙台,2005,2,0.828571,3.0,-1.3,4.335714,8.6,0.3,-2.221429,...,-4.7,2.000000,25.0,0.0,4.392857,9.0,0.0,68.750000,89.0,48.0
4,仙台,2005,3,4.061290,8.2,-1.2,8.332258,13.5,0.1,0.225806,...,-4.6,1.903226,28.0,0.0,5.522581,10.8,0.0,63.000000,89.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6715,鹿児島,2021,12,10.451613,14.5,3.9,15.135484,19.8,5.9,6.261290,...,0.2,1.548387,41.0,0.0,5.693548,9.8,0.0,67.516129,90.0,53.0
6716,鹿児島,2022,1,9.012903,14.4,6.1,13.500000,17.6,9.1,5.341935,...,0.2,1.451613,32.5,0.0,4.429032,10.0,0.0,63.903226,91.0,48.0
6717,鹿児島,2022,2,8.314286,13.8,5.2,12.721429,18.3,8.5,4.932143,...,1.2,0.821429,10.0,0.0,4.642857,10.7,0.0,62.428571,83.0,47.0
6718,鹿児島,2022,3,14.361290,20.0,9.4,19.477419,25.7,12.1,10.032258,...,4.1,6.000000,82.5,0.0,4.958065,11.1,0.0,71.064516,88.0,49.0


train.csv, test.csvには「各地」というエリアがあります。天候データ側ではこれを「全国」と読み替えることにして、全国平均を算出しておきます

In [7]:
agg_cols = [i for i in gb_df.columns if i not in ['year', 'month', 'area']]
tmp_df = gb_df.groupby(['year', 'month'])[agg_cols].agg(['mean']).reset_index()

new_cols = []
for col1, col2 in tmp_df.columns:
    new_cols.append(col1)

tmp_df.columns = new_cols
tmp_df['area'] = '全国'
tmp_df = tmp_df[gb_df.columns]
tmp_df

,area,year,month,mean_mean_temp,max_mean_temp,min_mean_temp,mean_max_temp,max_max_temp,min_max_temp,mean_min_temp,...,min_min_temp,mean_sum_rain,max_sum_rain,min_sum_rain,mean_sun_time,max_sun_time,min_sun_time,mean_mean_humid,max_mean_humid,min_mean_humid
0,全国,2004,11,13.046750,18.481250,8.406250,17.580375,22.950000,11.609375,9.046250,...,4.287500,3.456875,42.578125,0.0,5.446625,9.478125,0.000000,67.877500,87.96875,49.59375
1,全国,2004,12,8.011492,14.731250,1.662500,12.469456,21.259375,4.171875,4.086089,...,-1.271875,3.554940,54.796875,0.0,4.952319,9.059375,0.000000,65.214718,85.18750,48.43750
2,全国,2005,1,4.076210,7.975000,0.912500,8.123286,14.028125,3.909375,0.388609,...,-3.300000,2.198085,26.328125,0.0,4.467498,8.940625,0.006250,64.005040,84.96875,46.59375
3,全国,2005,2,4.199888,9.156250,0.143750,8.256920,14.600000,2.921875,0.549442,...,-3.390625,2.989955,25.125000,0.0,4.314951,9.321875,0.000000,63.796875,88.56250,46.25000
4,全国,2005,3,7.202218,12.068750,1.228125,11.891431,18.268750,4.409375,2.865927,...,-2.656250,3.122984,29.046875,0.0,5.416835,10.934375,0.000000,61.677419,87.09375,41.53125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,全国,2021,12,6.865927,12.515625,0.768750,11.254234,17.865625,3.565625,2.880746,...,-2.609375,2.730847,30.562500,0.0,5.176210,9.109375,0.062500,68.081653,89.53125,49.62500
206,全国,2022,1,4.046069,7.256250,0.859375,8.298690,12.337500,3.906250,0.276109,...,-3.653125,1.477319,20.171875,0.0,5.263206,9.231250,0.003125,64.570565,89.18750,49.43750
207,全国,2022,2,4.209933,8.478125,0.787500,8.639286,14.621875,3.737500,0.331696,...,-3.053125,1.676897,17.187500,0.0,5.717634,10.218750,0.012500,62.570312,87.65625,46.65625
208,全国,2022,3,9.924496,15.990625,4.509375,15.028427,22.528125,7.868750,5.475907,...,0.087500,3.296371,38.281250,0.0,5.885484,11.046875,0.009375,66.190524,89.40625,43.00000


都道府県別と全国平均の値を結合します

In [8]:
wea_df = pd.concat([gb_df, tmp_df])

卸売データと天候データを結合するには、各々でエリアの名称が異なるため、エリアの対応をとる必要があります<br>
エリアの変換を行うmapを作成します

In [9]:
all_df = pd.concat([train_df, test_df])
all_df

,kind,date,amount,mode_price,area
0,だいこん,20051107,201445.0,735.0,千葉
1,だいこん,20051108,189660.0,840.0,千葉_各地_青森
2,だいこん,20051110,218166.0,735.0,千葉_各地_青森
3,だいこん,20051111,182624.0,682.5,千葉_青森
4,だいこん,20051112,220691.0,682.5,千葉_青森
...,...,...,...,...,...
315,ミニトマト,20220526,NaN,NaN,各地_愛知
316,ミニトマト,20220527,NaN,NaN,各地_愛知
317,ミニトマト,20220528,NaN,NaN,各地_愛知
318,ミニトマト,20220530,NaN,NaN,各地_愛知


卸売データのエリアを列挙します

In [10]:
area_pairs = all_df['area'].unique()

yasai_areas = set()

for area_pair in area_pairs:
    areas = area_pair.split('_')
    yasai_areas |= set(areas)

yasai_areas

{'アメリカ',
 'カナダ',
 'トンガ',
 'ニュージーランド',
 'メキシコ',
 '中国',
 '佐賀',
 '兵庫',
 '北海道',
 '千葉',
 '各地',
 '和歌山',
 '埼玉',
 '宮城',
 '宮崎',
 '山形',
 '山梨',
 '岩手',
 '徳島',
 '愛媛',
 '愛知',
 '新潟',
 '東京',
 '栃木',
 '沖縄',
 '熊本',
 '神奈川',
 '福岡',
 '福島',
 '秋田',
 '群馬',
 '茨城',
 '長崎',
 '長野',
 '青森',
 '静岡',
 '香川',
 '高知',
 '鹿児島'}

天候データのエリアを列挙します

In [11]:
wea_areas = wea_df['area'].unique()
wea_areas

array(['仙台', '佐賀', '前橋', '千葉', '名古屋', '和歌山', '宇都宮', '宮崎', '山形', '帯広',
       '徳島', '新潟', '東京', '松山', '横浜', '水戸', '浜松', '熊本', '熊谷', '甲府', '盛岡',
       '神戸', '福岡', '福島', '秋田', '那覇', '長崎', '長野', '青森', '高松', '高知', '鹿児島',
       '全国'], dtype=object)

エリア変換のmapを作ります

In [12]:
area_map = {}

update_area_map = {
    '岩手':'盛岡','宮城':'仙台','静岡':'浜松','沖縄':'那覇','神奈川':'横浜','愛知':'名古屋','茨城':'水戸','北海道':'帯広','各地':'全国',
    '兵庫':'神戸','香川':'高松','埼玉':'熊谷','国内':'全国','山梨':'甲府','栃木':'宇都宮','群馬':'前橋','愛媛':'松山'
}

for yasai_area in yasai_areas:
    if yasai_area not in wea_areas and yasai_area not in update_area_map:
        area_map[yasai_area] = '全国' # 外国の天候は全国にしておく
    else:
        area_map[yasai_area] = yasai_area

area_map = {**area_map, **update_area_map}
area_map

{'東京': '東京',
 '千葉': '千葉',
 '愛知': '名古屋',
 '兵庫': '神戸',
 '新潟': '新潟',
 '青森': '青森',
 '埼玉': '熊谷',
 '愛媛': '松山',
 '佐賀': '佐賀',
 '栃木': '宇都宮',
 '長野': '長野',
 '福島': '福島',
 '徳島': '徳島',
 '北海道': '帯広',
 '山梨': '甲府',
 '沖縄': '那覇',
 '岩手': '盛岡',
 '山形': '山形',
 'トンガ': '全国',
 '香川': '高松',
 '福岡': '福岡',
 '和歌山': '和歌山',
 '神奈川': '横浜',
 '鹿児島': '鹿児島',
 '熊本': '熊本',
 '秋田': '秋田',
 '茨城': '水戸',
 'アメリカ': '全国',
 '高知': '高知',
 '宮城': '仙台',
 '各地': '全国',
 'ニュージーランド': '全国',
 '群馬': '前橋',
 '中国': '全国',
 '宮崎': '宮崎',
 '静岡': '浜松',
 '長崎': '長崎',
 'カナダ': '全国',
 'メキシコ': '全国',
 '国内': '全国'}

卸売データのエリアを天候データのエリアに変換します

In [13]:
all_df['area'] = all_df['area'].apply(lambda x: '_'.join([area_map[i] for i in x.split('_')]))
all_df

,kind,date,amount,mode_price,area
0,だいこん,20051107,201445.0,735.0,千葉
1,だいこん,20051108,189660.0,840.0,千葉_全国_青森
2,だいこん,20051110,218166.0,735.0,千葉_全国_青森
3,だいこん,20051111,182624.0,682.5,千葉_青森
4,だいこん,20051112,220691.0,682.5,千葉_青森
...,...,...,...,...,...
315,ミニトマト,20220526,NaN,NaN,全国_名古屋
316,ミニトマト,20220527,NaN,NaN,全国_名古屋
317,ミニトマト,20220528,NaN,NaN,全国_名古屋
318,ミニトマト,20220530,NaN,NaN,全国_名古屋


In [14]:
test_df = all_df.iloc[train_df.shape[0]:]
train_df = all_df.iloc[:train_df.shape[0]]

ここまで加工したデータを出力します（出力せず、このまま学習・推論を行っても構いません）

In [15]:
test_df.to_csv('../data/preprocessed_test.csv', index=False)
train_df.to_csv('../data/preprocessed_train.csv', index=False)

一方で天候データについても、卸売データと単純にmergeするには加工が必要です<br>「千葉_全国_青森」のように複数エリアに跨っている場合の各統計量について、各エリアの平均を取ることにします

In [16]:
area_pairs = all_df['area'].unique()

In [17]:
target_cols = [i for i in wea_df.columns if i != 'area']

In [18]:
area_pair_dfs = []

for area_pair in area_pairs:
    areas = area_pair.split('_')
    if len(areas) > 0:
        area = areas[0]
        base_tmp_df = wea_df[wea_df['area'] == area]
        base_tmp_df = base_tmp_df[target_cols].reset_index(drop=True)
        for area in areas[1:]:
            tmp_df = wea_df[wea_df['area'] == area]
            tmp_df = tmp_df[target_cols].reset_index(drop=True)
            base_tmp_df = base_tmp_df.add(tmp_df)
        base_tmp_df /= len(areas)
        base_tmp_df['area'] = area_pair
        area_pair_dfs.append(base_tmp_df)

In [19]:
area_pair_df = pd.concat(area_pair_dfs)
print(area_pair_df.shape)
area_pair_df.head()

(50820, 21)


,year,month,mean_mean_temp,max_mean_temp,min_mean_temp,mean_max_temp,max_max_temp,min_max_temp,mean_min_temp,max_min_temp,...,mean_sum_rain,max_sum_rain,min_sum_rain,mean_sun_time,max_sun_time,min_sun_time,mean_mean_humid,max_mean_humid,min_mean_humid,area
0,2004.0,11.0,14.596000,19.3,9.6,18.340000,22.2,12.4,11.228000,17.4,...,5.840000,87.0,0.0,5.084000,9.5,0.0,71.760000,93.0,43.0,千葉
1,2004.0,12.0,9.641935,19.0,2.2,13.564516,24.3,3.8,6.138710,11.5,...,2.435484,26.0,0.0,5.509677,9.4,0.0,58.612903,86.0,37.0,千葉
2,2005.0,1.0,6.277419,13.1,3.1,10.461290,17.2,5.2,2.522581,6.4,...,3.338710,66.5,0.0,6.612903,9.8,0.0,54.838710,89.0,32.0,千葉
3,2005.0,2.0,5.960714,12.0,3.3,9.864286,15.4,5.1,2.060714,4.6,...,1.910714,11.5,0.0,5.225000,9.7,0.0,55.714286,93.0,31.0,千葉
4,2005.0,3.0,8.800000,13.7,1.6,13.122581,18.4,3.6,4.596774,10.0,...,2.612903,22.5,0.0,5.529032,11.3,0.0,59.129032,91.0,31.0,千葉


ここまで加工したデータを出力します（出力せず、このまま学習・推論を行っても構いません）

In [20]:
area_pair_df.to_csv('../data/preprocessed_weather.csv', index=False)